In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction

## Setup

In [40]:
import pandas as pd
from urllib import request
from tqdm import tqdm
from pathlib import Path
import yaml

## Download metadata file from Encode

In [41]:
!wget -O metadata.tsv "https://www.encodeproject.org/metadata/?status=released&internal_tags=ENCORE&assay_title=eCLIP&biosample_ontology.term_name=K562&biosample_ontology.term_name=HepG2&files.file_type=bed+narrowPeak&type=Experiment&files.analyses.status=released"

--2024-11-08 11:19:28--  https://www.encodeproject.org/metadata/?status=released&internal_tags=ENCORE&assay_title=eCLIP&biosample_ontology.term_name=K562&biosample_ontology.term_name=HepG2&files.file_type=bed+narrowPeak&type=Experiment&files.analyses.status=released
Resolving www.encodeproject.org (www.encodeproject.org)... 34.211.244.144
Connecting to www.encodeproject.org (www.encodeproject.org)|34.211.244.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/tsv]
Saving to: ‘metadata.tsv’

metadata.tsv            [               <=>  ] 584.85K   129KB/s    in 4.5s    

2024-11-08 11:19:33 (129 KB/s) - ‘metadata.tsv’ saved [598890]



In [42]:
metadata = pd.read_csv('metadata.tsv', sep='\t')

In [43]:
metadata

,File accession,File format,File type,File format type,Output type,File assembly,Experiment accession,Assay,Donor(s),Biosample term id,...,Platform,Controlled by,File Status,s3_uri,Azure URL,File analysis title,File analysis status,Audit WARNING,Audit NOT_COMPLIANT,Audit ERROR
0,ENCFF535FFL,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR293FWY,eCLIP,/human-donors/ENCDO000AAD/,EFO:0002067,...,NaN,NaN,released,s3://encode-public/2022/01/12/ce2957c3-f1cd-41...,https://datasetencode.blob.core.windows.net/da...,Lab custom GRCh38 V29,released,NaN,NaN,NaN
1,ENCFF494PHH,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR293FWY,eCLIP,/human-donors/ENCDO000AAD/,EFO:0002067,...,NaN,NaN,released,s3://encode-public/2022/01/12/47750181-d073-44...,https://datasetencode.blob.core.windows.net/da...,Lab custom GRCh38 V29,released,NaN,NaN,NaN
2,ENCFF007VPS,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR293FWY,eCLIP,/human-donors/ENCDO000AAD/,EFO:0002067,...,NaN,NaN,released,s3://encode-public/2022/01/12/c4a5f910-0ad2-4c...,https://datasetencode.blob.core.windows.net/da...,Lab custom GRCh38 V29,released,NaN,NaN,NaN
3,ENCFF204THM,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR361OCV,eCLIP,/human-donors/ENCDO000AAC/,EFO:0001187,...,NaN,NaN,released,s3://encode-public/2018/04/29/106b9198-a9e2-4c...,https://datasetencode.blob.core.windows.net/da...,Lab custom GRCh38,released,NaN,NaN,NaN
4,ENCFF501GXB,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR361OCV,eCLIP,/human-donors/ENCDO000AAC/,EFO:0001187,...,NaN,NaN,released,s3://encode-public/2018/04/29/a93271e5-2ea5-48...,https://datasetencode.blob.core.windows.net/da...,Lab custom GRCh38,released,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,ENCFF645PLG,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR194HZU,eCLIP,/human-donors/ENCDO000AAC/,EFO:0001187,...,NaN,NaN,released,s3://encode-public/2016/11/30/987f23ff-4b5f-49...,https://datasetencode.blob.core.windows.net/da...,Lab custom GRCh38,released,NaN,NaN,NaN
746,ENCFF560WXZ,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR194HZU,eCLIP,/human-donors/ENCDO000AAC/,EFO:0001187,...,NaN,NaN,released,s3://encode-public/2016/11/30/a4aaf4b0-23c9-4d...,https://datasetencode.blob.core.windows.net/da...,Lab custom GRCh38,released,NaN,NaN,NaN
747,ENCFF597NVR,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR456ASB,eCLIP,/human-donors/ENCDO000AAD/,EFO:0002067,...,NaN,NaN,released,s3://encode-public/2018/12/03/b6fb6a4b-3334-45...,https://datasetencode.blob.core.windows.net/da...,Lab custom GRCh38,released,NaN,NaN,NaN
748,ENCFF518MSP,bed narrowPeak,bed,narrowPeak,peaks,GRCh38,ENCSR456ASB,eCLIP,/human-donors/ENCDO000AAD/,EFO:0002067,...,NaN,NaN,released,s3://encode-public/2016/11/30/6bbdb98d-dc8b-4a...,https://datasetencode.blob.core.windows.net/da...,Lab custom GRCh38,released,NaN,NaN,NaN


## Prepare directory structure

In [44]:
BASE_FILE_PATH = Path("./csv/")

# copied from https://stackoverflow.com/a/57892171
def rm_tree(pth: Path):
    for child in pth.iterdir():
        if child.is_file():
            child.unlink()
        else:
            rm_tree(child)
    pth.rmdir()

if BASE_FILE_PATH.exists():
    rm_tree(BASE_FILE_PATH)
    
BASE_FILE_PATH.mkdir()

In [45]:
!rm metadata.tsv

## Transform data

In [46]:
# get all unique protein names - all of our classes
targets = metadata['Experiment target'].unique()
targets

array(['DDX1-human', 'NIP7-human', 'FXR2-human', 'EEF2-human',
       'DDX43-human', 'DDX47-human', 'RPS6-human', 'ELAC2-human',
       'FMR1-human', 'SRSF9-human', 'NSUN2-human', 'TARDBP-human',
       'RPS10-human', 'MORC2-human', 'RNF187-human', 'SRSF1-human',
       'SSB-human', 'PUM1-human', 'EIF4E-human', 'EXOSC10-human',
       'FXR1-human', 'MBNL1-human', 'TIAL1-human', 'NIPBL-human',
       'MATR3-human', 'SUPV3L1-human', 'STAU2-human', 'ZC3H8-human',
       'UPF1-human', 'SDAD1-human', 'ZNF800-human', 'SAFB-human',
       'FUS-human', 'BCLAF1-human', 'DDX52-human', 'CPEB4-human',
       'PABPN1-human', 'KHSRP-human', 'UTP3-human', 'HNRNPC-human',
       'PCBP1-human', 'G3BP1-human', 'APOBEC3C-human', 'AKAP1-human',
       'EIF3G-human', 'UTP18-human', 'HNRNPK-human', 'METTL1-human',
       'AATF-human', 'DDX21-human', 'ELAVL1-human', 'YBX3-human',
       'HLTF-human', 'RYBP-human', 'WDR43-human', 'AQR-human',
       'WDR3-human', 'GRWD1-human', 'APEX1-human', 'ADAT1-human',
 

In [ ]:
BED_HEADER = ['chr','start','end','name','score','strand','signalValue','pValue','qValue','peak']
COLUMNS_OF_INTEREST = ['chr', 'start', 'end', 'strand','pValue']
ALLOWED_CHROMOSOMES = ['chr' + str(i) for i in range(1, 22)] + ['chrX', 'chrY', 'chrMT']

coord_links = []

for target in tqdm(targets):

  # iterate over all experiments for current protein
  for experiment in metadata[metadata['Experiment target'] == target]['Experiment accession'].unique():
    
    protein_name = target[: target.index('-')]
    rows = metadata[(metadata['Experiment accession'] == experiment) & (metadata['Experiment target'] == target)]

    if (len(rows) > 1):
      # we are in a situation with multiple replicates

      # look at values in column Biological replicate(s) and choose the one where replicate numbers are merged
      rows = rows[rows['Biological replicate(s)'].str.contains(',')]  # keep only rows with multiple replicates

    # collect all peaks for current protein
    protein_reads = []
    for index, row in rows.iterrows():

      local_file = row['File accession'] + '.bed.gz'

      # downloading files in while-try, because network connection might break sometimes
      success = False
      while not success:
        try:
          request.urlretrieve(row['File download URL'], local_file)
          success = True
        except:
          print('Problem with file ', local_file, '. Trying again.')
      coord_links.append(row['File download URL'])
      reads = pd.read_csv(local_file, sep='\t', compression='gzip', header=None, names=BED_HEADER)
      
      # keep just necessary columns
      reads = reads[COLUMNS_OF_INTEREST]
      # keep just sequences from chromosomes chr1 - ch22, chrX, chrX and chrMT
      reads = reads[reads['chr'].isin(ALLOWED_CHROMOSOMES)]
      # removing outliers - keep just sequences >= 20 bp and <= 100 bp
      reads = reads[((reads['end'] - reads['start']) >= 20) & ((reads['end'] - reads['start']) <= 100)]

      # add peak from current file to peaks from previous file
      protein_reads.extend(reads.values.tolist())
      
      # cleaning - delete downloaded bed file
      Path(local_file).unlink()

  df = pd.DataFrame(protein_reads, columns=COLUMNS_OF_INTEREST)

  # add column with name of protein
  df['protein_name'] = protein_name

  filename = protein_name + '.csv.gz'
  df.to_csv(BASE_FILE_PATH / filename, index=False, compression='gzip')

with open(BASE_FILE_PATH / 'coord_links.txt', 'w') as f:
  f.write('\n'.join(coord_links))

100%|██████████| 168/168 [04:20<00:00,  1.55s/it]


## YAML file

In [48]:
# YAML file with metadata
# we store paths of reference fasta files

desc = {
  'version': 0,
  'classes': {}
}
for target in targets:
  name = target[: target.index('-')]
  desc['classes'][name] = {
      'type': 'fa.gz',
      'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
      'extra_processing': 'ENSEMBL_HUMAN_GENOME'
  }

with open(BASE_FILE_PATH / 'metadata.yaml', 'w') as fw:    
    yaml.dump(desc, fw)

desc

{'version': 0,
 'classes': {'DDX1': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'NIP7': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'FXR2': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'EEF2': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'DDX43': {'type': 'fa.gz',
   'url': 'http://ftp.ensembl.org/pub/release-97/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.toplevel.fa.gz',
   'extra_processing': 'ENSEMBL_HUMAN_GENOME'},
  'DDX47': {'type': 'fa.gz',